In [11]:
! git clone https://github.com/HakimTem/MusicAI.git
! it pull

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time
import functools
from scipy.io.wavfile import read


is_correct_tf_version = '1.13.1' in tf.__version__
assert is_correct_tf_version, "Wrong tensorflow version ({}) installed".format(tf.__version__)

is_eager_enabled = tf.executing_eagerly()
assert is_eager_enabled,      "Tensorflow eager mode is not enabled"

a = read("adios.wav")








fatal: destination path 'MusicAI' already exists and is not an empty directory.
/bin/bash: it: command not found
